In [1]:
import graphlab as gl
import pyLDAvis 
import pyLDAvis.graphlab
import pandas as pd
import ftfy
import numpy as np

A newer version of GraphLab Create (v1.7.1) is available! Your current version is v1.6.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.
[INFO] This non-commercial license of GraphLab Create is assigned to kimitaka01@gmail.com and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-29632 - Server binary: /Users/kimitakanakazawa/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1449981498.log
[INFO] GraphLab Server Version: 1.6.1


# Goal:
- Find who the hosts are on airbnb

In [2]:
#Automatic Rendering of Visualisation
pyLDAvis.enable_notebook()

In [3]:
df = pd.read_csv('data/listings_new.csv')
print df.columns
df.head(1)

Index([u'Unnamed: 0', u'id', u'listing_url', u'name', u'summary', u'space',
       u'description', u'neighborhood_overview', u'notes', u'transit',
       u'host_id', u'host_url', u'host_name', u'host_since', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_listings_count', u'street',
       u'neighbourhood', u'zipcode', u'latitude', u'longitude',
       u'property_type', u'room_type', u'accommodates', u'bathrooms',
       u'bedrooms', u'beds', u'bed_type', u'amenities', u'square_feet',
       u'price', u'weekly_price', u'monthly_price', u'security_deposit',
       u'cleaning_fee', u'guests_included', u'extra_people', u'minimum_nights',
       u'maximum_nights', u'number_of_reviews', u'review_scores_rating',
       u'review_scores_accuracy', u'review_scores_cleanliness',
       u'review_scores_checkin', u'review_scores_communication',
       u'review_scores_location', u'review_scores_value', u'instant_bookab

,Unnamed: 0,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
0,0,1162609,https://www.airbnb.com/rooms/1162609,Lovely One Bedroom Apartment,This is a big well designed space. Our neighbo...,This is a great one bedroom apartment. It is a...,This is a big well designed space. Our neighbo...,NaN,NaN,NaN,...,89,9,9,10,10,9,9,f,strict,0.28


In [4]:
about_host = df['host_about']
about_host = pd.DataFrame(about_host[about_host.notnull() == True])
about_host.head(1)

,host_about
0,I am from SF.


In [5]:
about_host_gl = gl.SFrame(about_host)
# about_host_gl.show()

# unigram

In [6]:
# unigram
about_host_gl['features'] = gl.text_analytics.count_ngrams(about_host_gl['host_about'],1).dict_trim_by_keys(gl.text_analytics.stopwords(), True)
about_host_gl

host_about,features
I am from SF.,{'sf': 1}
Retired AccountingProfessional. Down to ...,"{'enjoy': 1, 'life': 2,'outdoor': 1, ..."
i'm a software engineerthat works on Google ...,"{'engineer': 1,'software': 1, 'works': ..."
I like experiencing theauthentic culture of the ...,"{'enjoy': 1, 'limited':1, 'learning': 1, ..."
"I enjoy seeing livemusic, traveling, ...","{'enjoy': 1, 'surfing':1, 'brewing': 1, 'live': ..."
San Francisco natives whoown this home. Alexis ...,"{'love': 2, 'san': 1,'people': 1, 'traveli ..."
"Born and bred in London,I fell in love with ...","{'work': 1, 'love': 2,'san': 1, 'random': 1, ..."
"i love to connect withand empower people, ...","{'coach': 1, 'love': 1,'people': 1, ..."
My on going love affairwith San Francisco made ...,"{'enjoy': 1, 'affair': 1,'walking': 1, 'love': 2, ..."
I moved to SF afterliving in the East Bay ...,"{'enjoy': 1, 'lands': 1,'ve': 2, 'bogota': 1, ..."


In [7]:
#Creating Topic Model
#gl.topic_model.create?
topicModel=gl.topic_model.create(about_host_gl['features'], num_topics=3, num_iterations=50)

PROGRESS: Learning a topic model
PROGRESS:        Number of documents      5454
PROGRESS:            Vocabulary size     12190
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 359.07ms      | 4.80437e+06    | 0               |
PROGRESS: | 20        | 679.816ms     | 5.21781e+06    | 0               |
PROGRESS: | 30        | 1.02s         | 4.4291e+06     | 0               |
PROGRESS: | 40        | 1.37s         | 4.76992e+06    | 0               |
PROGRESS: | 50        | 1.70s         | 4.69152e+06    | 0               |
PROGRESS: +-----------+---------------+----------------+-----------------+


In [8]:
pyLDAvis.graphlab.prepare(topicModel, about_host_gl['features'])

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      34.461762        1       1 -0.159688  0.271766
0      33.058219        1       2 -0.163284 -0.269756
1      32.480019        1       3  0.322973 -0.002010, topic_info=      Category         Freq        Term  Total  loglift  logprob
839    Default  3956.000000        love   3956  30.0000  30.0000
1078   Default  3808.000000         san   3808  29.0000  29.0000
5881   Default  3612.000000   francisco   3612  28.0000  28.0000
2822   Default  1514.000000       years   1514  27.0000  27.0000
8603   Default   949.000000      airbnb    949  26.0000  26.0000
4601   Default   914.000000          ve    914  25.0000  25.0000
3107   Default  1078.000000        work   1078  24.0000  24.0000
1038   Default  1629.000000        city   1629  23.0000  23.0000
608    Default  1714.000000      people   1714  22.0000  22.0000
6014   Default   741.000000     friends    741  21.0000  21.0000
6322   Default  1481.000000       enjoy   1481  20.0000  20.0000
4591   Default   815.000000      family    815  19.0000  19.0000
135    Default  1875.000000      travel   1875  18.0000  18.0000
4424   Default   672.000000        area    672  17.0000  17.0000
5884   Default  1733.000000        home   1733  16.0000  16.0000
5683   Default   621.000000      places    621  15.0000  15.0000
7064   Default   929.000000      living    929  14.0000  14.0000
8976   Default   558.000000        host    558  13.0000  13.0000
5633   Default   948.000000          sf    948  12.0000  12.0000
90     Default   567.000000       music    567  11.0000  11.0000
1383   Default   556.000000    favorite    556  10.0000  10.0000
11507  Default   913.000000   traveling    913   9.0000   9.0000
12145  Default   491.000000        year    491   8.0000   8.0000
1632   Default   504.000000   beautiful    504   7.0000   7.0000
7739   Default   469.000000       share    469   6.0000   6.0000
3844   Default  1063.000000        live   1063   5.0000   5.0000
1328   Default  1155.000000       world   1155   4.0000   4.0000
11124  Default   409.000000         art    409   3.0000   3.0000
4479   Default  1114.000000        time   1114   2.0000   2.0000
10973  Default   406.000000        feel    406   1.0000   1.0000
...        ...          ...         ...    ...      ...      ...
7750    Topic3   165.790160  technology    166   1.0724  -5.9077
938     Topic3   164.790162        avid    165   1.0724  -5.9137
197     Topic3   161.790169      hidden    162   1.0724  -5.9321
5157    Topic3   157.790180       works    158   1.0725  -5.9571
67      Topic3   153.790190        kids    154   1.0725  -5.9827
7390    Topic3   151.790196     running    152   1.0725  -5.9957
10167   Topic3   151.790196     website    152   1.0725  -5.9957
9861    Topic3   150.790199       check    151   1.0725  -6.0023
9545    Topic3   189.688613   countries    191   1.0671  -5.7727
4591    Topic3   779.619739      family    815   1.0319  -4.3570
3107    Topic3   999.933374        work   1078   0.9968  -4.1124
683     Topic3   434.455374       happy    461   1.0186  -4.9401
2750    Topic3   349.831412       moved    367   1.0247  -5.1621
4430    Topic3   241.821079         lot    253   1.0278  -5.5309
11507   Topic3   718.131671   traveling    913   0.8524  -4.4230
3844    Topic3   822.177662        live   1063   0.8317  -4.2915
608     Topic3  1189.588799      people   1714   0.7360  -3.9095
8497    Topic3   296.879312      native    334   0.9548  -5.3262
6109    Topic3   188.503356      active    195   1.0411  -5.7780
11151   Topic3   203.547975    recently    218   1.0108  -5.6968
8065    Topic3   445.826991     meeting    683   0.6790  -4.8867
6594    Topic3   431.136384        make    649   0.6627  -4.9539
10182   Topic3   420.130795       place    633   0.6619  -4.9798
1784    Topic3   268.926635       guest    328   0.8739  -5.4252
1328    Topic3   546.416828       world   1155 

## They are...
- married couples
- love traveling
- working professonal in something

# Bigram

In [9]:
about_host_gl['features'] = gl.text_analytics.count_ngrams(about_host_gl['host_about'],2).dict_trim_by_keys(gl.text_analytics.stopwords(), True)
topicModel=gl.topic_model.create(about_host_gl['features'], num_topics=3, num_iterations=50)
pyLDAvis.graphlab.prepare(topicModel, about_host_gl['features'])

PROGRESS: Learning a topic model
PROGRESS:        Number of documents      5454
PROGRESS:            Vocabulary size     89435
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 822.993ms     | 5.24138e+06    | 0               |
PROGRESS: | 20        | 1.48s         | 5.28459e+06    | 0               |
PROGRESS: | 30        | 2.15s         | 5.36633e+06    | 0               |
PROGRESS: | 40        | 2.84s         | 5.50356e+06    | 0               |
PROGRESS: | 50        | 3.51s         | 5.02187e+06    | 0               |
PROGRESS: +-----------+---------------+----------------+-----------------+


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      34.560060        1       1  0.230796 -0.170202
0      33.211053        1       2 -0.269605 -0.105951
1      32.228887        1       3  0.038809  0.276153, topic_info=      Category         Freq             Term  Total  loglift  logprob
38222  Default  3609.000000    san francisco   3609  30.0000  30.0000
71642  Default  1179.000000          love to   1179  29.0000  29.0000
58505  Default   842.000000             am a    842  28.0000  28.0000
76992  Default  2146.000000              i m   2146  27.0000  27.0000
73124  Default   720.000000         lived in    720  26.0000  26.0000
34365  Default  1759.000000           i love   1759  25.0000  25.0000
51274  Default  2201.000000             i am   2201  24.0000  24.0000
80061  Default   912.000000           we are    912  23.0000  23.0000
86312  Default  1165.000000           i have   1165  22.0000  22.0000
67986  Default  1617.000000           in san   1617  21.0000  21.0000
18062  Default   623.000000        living in    623  20.0000  20.0000
22574  Default   629.000000          we love    629  19.0000  19.0000
69718  Default   590.000000         the city    590  18.0000  18.0000
50036  Default   499.000000         bay area    499  17.0000  17.0000
76342  Default  1892.000000           in the   1892  16.0000  16.0000
79372  Default   523.000000           have a    523  15.0000  15.0000
5582   Default   385.000000          the bay    385  14.0000  14.0000
8060   Default   388.000000       around the    388  13.0000  13.0000
32745  Default   397.000000          we have    397  12.0000  12.0000
71580  Default   428.000000          and the    428  11.0000  11.0000
31270  Default   909.000000        the world    909  10.0000  10.0000
58572  Default   370.000000       forward to    370   9.0000   9.0000
23249  Default   369.000000            i can    369   8.0000   8.0000
86167  Default   478.000000             as a    478   7.0000   7.0000
62053  Default   357.000000  originally from    357   6.0000   6.0000
71701  Default   343.000000           when i    343   5.0000   5.0000
33326  Default   328.000000          my home    328   4.0000   4.0000
7168   Default   757.000000             i ve    757   3.0000   3.0000
79694  Default   325.000000           i live    325   2.0000   2.0000
59052  Default   396.000000            in sf    396   1.0000   1.0000
...        ...          ...              ...    ...      ...      ...
62402   Topic3   107.806927           by the    108   1.0863  -7.0563
31711   Topic3   107.806927         my place    108   1.0863  -7.0563
17710   Topic3   105.806937           what i    106   1.0863  -7.0749
10280   Topic3   104.806942         would be    105   1.0863  -7.0844
17073   Topic3   101.806958       live music    102   1.0863  -7.1134
69426   Topic3   100.806963           i want    101   1.0863  -7.1232
50330   Topic3   123.862494         and work    125   1.0781  -6.9182
32188   Topic3   202.873905          grew up    209   1.0567  -6.4256
4935    Topic3   375.109060           i like    405   1.0087  -5.8121
28933   Topic3   121.819236         was born    123   1.0780  -6.9345
86167   Topic3   427.298584             as a    478   0.9728  -5.6823
14942   Topic3   337.948459      francisco i    380   0.9626  -5.9219
13407   Topic3   179.928212         food and    191   1.0266  -6.5458
67293   Topic3   273.475744          work in    306   0.9679  -6.1332
38222   Topic3  2016.393817    san francisco   3609   0.4785  -4.1550
34365   Topic3  1050.229698           i love   1759   0.5657  -4.7865
76342   Topic3  1081.216455           in the   1892   0.5051  -4.7742
67986   Topic3   873.242598           in san   1617   0.4436  -4.9928
31270   Topic3   512.208193        the world    909   0.5076  -5.5047
82886   Topic3   245.867220      my favorite    314   0.8294  -6.2459
76992   Topic3   818.844315              i m   